In [1]:
import sys
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
import numpy as np

# Add src to path
sys.path.append('src')

from datasets.classification import get_mnist_dataloaders, infer_dataset_info
from models.base_bnn import create_mlp
from models.standard_bnn import StandardBNN

print("🔍 Verifying StandardBNN.evaluate() function...")
print("=" * 60)


🔍 Verifying StandardBNN.evaluate() function...


## 1. Load MNIST Dataset and Verify Test Set Size


In [3]:
# Load MNIST dataset
print("📊 Loading MNIST dataset...")
train_loader, val_loader, test_loader = get_mnist_dataloaders(
    batch_size=64, 
    num_workers=5,
)

# Get dataset info
dataset_info = infer_dataset_info(train_loader)
print(f"Dataset info: {dataset_info}")

# Verify test set size
test_dataset_size = len(test_loader.dataset)
test_batches = len(test_loader)
print(f"Test set: {test_dataset_size} samples in {test_batches} batches")


📊 Loading MNIST dataset...
Dataset info: {'input_shape': (784,), 'input_size': 784, 'num_classes': 10}
Test set: 10000 samples in 157 batches


## 2. Create and Train BNN Model


In [6]:
# Create model
print("\n🏗️  Creating BNN model...")
model = create_mlp(
    input_size=dataset_info['input_size'],
    hidden_sizes=[128, 128],
    num_classes=dataset_info['num_classes']
)

bnn = StandardBNN(
    model=model,
    num_classes=dataset_info['num_classes'],
    prior_std=1.0,
    device='cuda'  # Use CPU for faster testing
)

# Quick training (just a few epochs for verification)
print("\n🚀 Training BNN (quick test)...")
bnn.fit(
    train_loader=train_loader,
    num_epochs=5,  # Very short training
    lr=1e-3,
    num_burn_in=2,  # Very short burn-in
    verbose=True
)

print(f"Collected {len(bnn.posterior_samples)} posterior samples")



🏗️  Creating BNN model...
Using GPU: NVIDIA GeForce RTX 4090
BNN initialized on device: cuda

🚀 Training BNN (quick test)...


Training BNN:   0%|          | 0/5 [00:00<?, ?it/s]

Training BNN:  40%|████      | 2/5 [00:28<00:42, 14.18s/it, Samples Collected=0, Burn-in=1/2]


OutOfMemoryError: CUDA out of memory. Tried to allocate 2.00 MiB. GPU 0 has a total capacity of 23.65 GiB of which 3.50 MiB is free. Process 1676362 has 18.80 GiB memory in use. Including non-PyTorch memory, this process has 4.83 GiB memory in use. Of the allocated memory 4.37 GiB is allocated by PyTorch, and 7.58 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)